# Sum together multiple 4D STEM scans

In [1]:
%matplotlib widget

from pathlib import Path

import h5py
import numpy as np
import matplotlib.pyplot as plt
import imageio

import ncempy.algo as nalgo
import ncempy.io as nio

import stempy.io as stio
import stempy.image as stim


In [123]:
# Load a sparse 4D camera data set

# Close all previous windows to avoid too many windows
plt.close('all')

scan_nums = (110, 111, 112, 113)
threshold = 4.5

all = []
for scan_num in scan_nums:
    dPath = Path('/mnt/hdd1/2020.11.30_Au')
    fPath = Path('data_scan{}_th{}_electrons_centered.h5'.format(scan_num, threshold))

    fname = dPath / fPath

    electron_events = stio.load_electron_counts(str(fname))
    
    all.append(electron_events.data)
    
    print('File: {}'.format(fname))
    print('Initial scan dimensions = {}'.format(electron_events.scan_dimensions))

File: /mnt/hdd1/2020.11.30_Au/data_scan110_th4.5_electrons_centered.h5
Initial scan dimensions = [257, 1024]
File: /mnt/hdd1/2020.11.30_Au/data_scan111_th4.5_electrons_centered.h5
Initial scan dimensions = [257, 1024]
File: /mnt/hdd1/2020.11.30_Au/data_scan112_th4.5_electrons_centered.h5
Initial scan dimensions = [257, 1024]
File: /mnt/hdd1/2020.11.30_Au/data_scan113_th4.5_electrons_centered.h5
Initial scan dimensions = [257, 1024]


In [124]:
out = []
for ii in range(all[0].shape[0]):
    tt = np.hstack([pp[ii] for pp in all])
    out.append(tt)
events_summed = np.array([np.array(x, copy=False) for x in out], dtype=np.object)

In [125]:
dp = stim.calculate_sum_sparse(events_summed[100], (576, 576))
dp0 = stim.calculate_sum_sparse(all[0][100], (576, 576))
dp1 = stim.calculate_sum_sparse(all[1][100], (576, 576))

from matplotlib.colors import LogNorm
fg,ax = plt.subplots(3, 1, sharex=True,sharey=True)
ax[0].imshow(dp+1,norm=LogNorm())
ax[1].imshow(dp0+1,norm=LogNorm())
ax[2].imshow(dp1+1,norm=LogNorm())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [127]:
import h5py


events = events_summed
scan_dimensions = electron_events.scan_dimensions
frame_dimensions = electron_events.frame_dimensions

with h5py.File(fname.with_name('battery_summed.h5'), 'w') as f:
    group = f.require_group('electron_events')
    scan_positions = group.create_dataset('scan_positions', (events.shape[0],), dtype=np.int32)
    # For now just assume we have all the frames, so the event index can
    # be used to derive the scan_postions.
    # TODO: This should be passed to use
    scan_positions[...] = [i for i in range(0, events.shape[0])]
    scan_positions.attrs['Nx'] = scan_dimensions[0]
    scan_positions.attrs['Ny'] = scan_dimensions[1]

    coordinates_type = h5py.special_dtype(vlen=np.uint32)
    frames = group.create_dataset('frames', (events.shape[0],), dtype=coordinates_type)
    # Add the frame dimensions as attributes
    frames.attrs['Nx'] = frame_dimensions[0]
    frames.attrs['Ny'] = frame_dimensions[1]

    frames[...] = events